### Dump all the images to see how I can format them


In [104]:
import pymupdf
from pathlib import Path
from PIL import Image
import io


exam_name = "2024_YKS_TYT"

doc_path = Path("..") /   "data_raw" / f"{exam_name}.pdf"

doc = pymupdf.open(doc_path)

for page_num in range(len(doc)):

    print(f"Page {page_num}")

    page = doc[page_num]

    image_list = page.get_images(full=True)
    
    for img_index, img in enumerate(image_list):
        xref = img[0]
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]
        image_ext = base_image["ext"]

        image = Image.open(io.BytesIO(image_bytes))

        # Save the image to a file
        image_path = Path("..") / "image_dump" / exam_name / f"page_{page_num}_img_{img_index}.{image_ext}"
        image.save(image_path)


Page 0
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20
Page 21
Page 22
Page 23
Page 24
Page 25
Page 26
Page 27
Page 28
Page 29
Page 30
Page 31
Page 32
Page 33
Page 34
Page 35
Page 36
Page 37
Page 38
Page 39
Page 40
Page 41
Page 42


In [105]:
import pymupdf
from pathlib import Path


exam_name = "2024_YKS_TYT"

doc_path = Path("..") /   "data_raw" / f"{exam_name}.pdf"

doc = pymupdf.open(doc_path)


for page_num, page in enumerate(doc):
    print(f"Page {page_num}")

    text = page.get_text("text")

    print(text)


Page 0
T.C. Ölçme, Seçme ve Yerleştirme Merkezi
Bu testlerin her hakkı saklıdır. Hangi amaçla olursa olsun, testlerin tamamının veya bir kısmının Merkezimizin yazılı izni olmadan kopya edilmesi, 
fotoğrafının çekilmesi, herhangi bir yolla çoğaltılması, yayımlanması ya da kullanılması yasaktır. Bu yasağa uymayanlar gerekli cezai sorumluluğu ve 
testlerin hazırlanmasındaki mali külfeti peşinen kabullenmiş sayılır.
YÜKSEKÖĞRETİM KURUMLARI SINAVI
TEMEL YETERLİLİK TESTİ
(2024-TYT)
8 HAZİRAN 2024 CUMARTESİ

Page 1
 
 
 
 Ö S Y M 
Bu soruların telif hakları ÖSYM’ye aittir. Sorular ÖSYM’nin yazılı izni olmaksızın    
hiçbir kişi, kurum veya kuruluş tarafından kullanılamaz. 
 

Page 2
2024-TYT/TÜR
TÜRKÇE TESTİ
1. Bu testte 40 soru vardır.
2. Cevaplarınızı, cevap kâğıdının Türkçe Testi için ayrılan kısmına işaretleyiniz.
Araştırmacılar doğada bulunmanın sağlığa olan
faydalarına işaret ediyor (dikkat çekiyor). Açık 
                          I
alanlarda vakit geçirmenin ruh sağlığı üzerinde 
koru

# Basic Data Dictionary


Question_Object
- Year
- Exam
- SubExam
- Section
- QuestionNumber
- Page
- ImageCount
- HasImage
- MaxImageWidth
- MaxImageHeight
- Passage
- Question
- Choices
- Q_Language
- Answer

Image_Object
- Year
- Exam
- Section
- QuestionNumber
- Page
- ImageHeight
- ImageWidth
- ImageOCR
- T_Language
- ImageDescription
- D_Language

# Helper Methods


### To check if something is the begging of a question


### Extract the question number


### Split the question and the choices


In [106]:
import re



# Function to check if a string is in the form of a number
def is_question_index(text):
    # Define the regex pattern
    pattern = r'^\d+\.$'
    return bool(re.match(pattern, text))

def extract_numbers(text):
    # Regular expression to extract the first two numbers

    pattern = r'\b(\d+)\b'

    numbers = re.findall(pattern, text)

    return numbers


def split_question_choices(text):

    pattern = r'\?\s*[\r\n]*\s*'


    # Split the text into parts based on the pattern
    parts = re.split(pattern, text)
    
    return parts


def has_choices(text, expected_choices=5):

    pattern = r'[A-E]\)'

    return expected_choices == len(re.findall(pattern, text))




# Road Map


1- Split the page into two <br>
2- Sort all the elements within the right and left. <br>
3- Find the beginning of the questions <br>


In [107]:
import pymupdf
from pathlib import Path

#Json 
import json


exam_name = "2024_YKS_TYT"

exam_year = exam_name.split("_")[0]
exam_type = exam_name.split("_")[1]
exam_subtype = exam_name.split("_")[2]


doc_path = Path("..") /   "data_raw" / f"{exam_name}.pdf"

doc = pymupdf.open(doc_path)

question_list = []
current_section = ""

multi_context_question_text = ""
multi_context_question_set = []
multi_context = False

for page_num in range(len(doc)):
        
        page = doc.load_page(page_num)  

        # Extract words as tuples (x0, y0, x1, y1, "text", block_no, line_no, word_no)
        blocks = page.get_text("blocks")

        right_blocks = []
        left_blocks = []

        page_width = page.rect.width

        for block in blocks:
            if block[0] < page_width / 2:
                # Block is on the left side of the page
                my_text_block= (block[0], block[1], block[2], block[3], block[4], block[5], block[6], 0)
                left_blocks.append(my_text_block)
            else:
                my_text_block= (block[0], block[1], block[2], block[3], block[4], block[5], block[6], 0)
                right_blocks.append(my_text_block)



        image_list = page.get_images(full=True)

        for image_ind, img_info in enumerate(image_list):
            xref = img_info[0]
            
            positions = page.get_image_rects(xref)
            

            height = positions[0][3] - positions[0][1]
            width = positions[0][2] - positions[0][0]

            base_image = doc.extract_image(xref)
            image_ext = base_image["ext"]

            my_img_obj = (positions[0][0], positions[0][1], positions[0][2], positions[0][3], xref, (height, width, image_ext), image_ind, 1)

            if my_img_obj[0] < page_width / 2:
                left_blocks.append(my_img_obj)
            else:
                right_blocks.append(my_img_obj)


        # Sort the blocks by their y-coordinates (top to bottom) and then by x-coordinates (left to right)
        left_blocks.sort(key=lambda x: (x[1], x[0]))
        right_blocks.sort(key=lambda x: (x[1], x[0]))

        # Combine the sorted blocks into a single list
        combined_blocks = left_blocks + right_blocks

        # print as json

        for block in combined_blocks:
            if block[7] == 0:
                print(f"TextBlock:<{block[4]}>")
            else:
                print(f"ImageBlock:<{block[4]}>")
                

TextBlock:<T.C. Ölçme, Seçme ve Yerleştirme Merkezi
>
TextBlock:<YÜKSEKÖĞRETİM KURUMLARI SINAVI
>
TextBlock:<8 HAZİRAN 2024 CUMARTESİ
>
TextBlock:<Bu testlerin her hakkı saklıdır. Hangi amaçla olursa olsun, testlerin tamamının veya bir kısmının Merkezimizin yazılı izni olmadan kopya edilmesi, 
fotoğrafının çekilmesi, herhangi bir yolla çoğaltılması, yayımlanması ya da kullanılması yasaktır. Bu yasağa uymayanlar gerekli cezai sorumluluğu ve 
testlerin hazırlanmasındaki mali külfeti peşinen kabullenmiş sayılır.
>
TextBlock:<TEMEL YETERLİLİK TESTİ
>
TextBlock:<(2024-TYT)
>
ImageBlock:<12>
TextBlock:< Ö S Y M 
>
TextBlock:<Bu soruların telif hakları ÖSYM’ye aittir. Sorular ÖSYM’nin yazılı izni olmaksızın    
>
TextBlock:<hiçbir kişi, kurum veya kuruluş tarafından kullanılamaz. 
 
>
TextBlock:< 
>
TextBlock:< 
>
TextBlock:< 
>
TextBlock:<2024-TYT/TÜR
TÜRKÇE TESTİ
>
TextBlock:<1. Bu testte 40 soru vardır.
>
TextBlock:<2. Cevaplarınızı, cevap kâğıdının Türkçe Testi için ayrılan kısmına işaret

In [173]:
import pymupdf
from pathlib import Path

#Json 
import json


exam_name = "2024_YKS_TYT"

exam_year = exam_name.split("_")[0]
exam_type = exam_name.split("_")[1]
exam_subtype = exam_name.split("_")[2]


doc_path = Path("..") /   "data_raw" / f"{exam_name}.pdf"

doc = pymupdf.open(doc_path)

question_list = []
current_section = ""

multi_context_question_text = ""
multi_context_question_set = []
multi_context = False

answer_key = False

for page_num in range(len(doc)):
        
        page = doc.load_page(page_num)  

        # Extract words as tuples (x0, y0, x1, y1, "text", block_no, line_no, word_no)
        blocks = page.get_text("blocks")

        right_blocks = []
        left_blocks = []

        page_width = page.rect.width

        for block in blocks:
            if block[0] < page_width / 2:
                # Block is on the left side of the page
                my_text_block= (block[0], block[1], block[2], block[3], block[4], block[5], block[6], 0)
                left_blocks.append(my_text_block)
            else:
                my_text_block= (block[0], block[1], block[2], block[3], block[4], block[5], block[6], 0)
                right_blocks.append(my_text_block)



        image_list = page.get_images(full=True)

        for image_ind, img_info in enumerate(image_list):
            xref = img_info[0]
            
            positions = page.get_image_rects(xref)
            

            height = positions[0][3] - positions[0][1]
            width = positions[0][2] - positions[0][0]

            base_image = doc.extract_image(xref)
            image_ext = base_image["ext"]

            my_img_obj = (positions[0][0], positions[0][1], positions[0][2], positions[0][3], xref, (height, width, image_ext), image_ind, 1)

            if my_img_obj[0] < page_width / 2:
                left_blocks.append(my_img_obj)
            else:
                right_blocks.append(my_img_obj)


        # Sort the blocks by their y-coordinates (top to bottom) and then by x-coordinates (left to right)
        left_blocks.sort(key=lambda x: (x[1], x[0]))
        right_blocks.sort(key=lambda x: (x[1], x[0]))

        # Combine the sorted blocks into a single list
        combined_blocks = left_blocks + right_blocks

        
        current_question = {}

        if not answer_key:
                for block in combined_blocks:
                    if block[7] == 0:
                        block_text = block[4]

                        #Check if the block is empty
                        if block_text.strip() == "":
                            continue

                        # If it is the beginning of a new chapter (correct way to check multiple strings)
                        if any(phrase in block_text for phrase in ["1. Bu testte", "2. Cevaplarınızı", "Ö S Y M", 
                            "Diğer sayfaya geçiniz.", "Bu soruların telif hakları", "hiçbir kişi, kurum veya kuruluş", "soruları Din Kültürü ve Ahlak Bilgisi", "TESTİ BİTTİ."]):
                            continue

                        if "CEVAPLARINIZI KONTROL EDİNİZ" in block_text:
                            # If it's the answer key, set the flag to True
                            answer_key = True
                            continue

                        if "soruları aşağıdaki parçaya göre" in block_text:
                            # If it's a multi-context question, save the text and continue
                            current_question = {}
                            multi_context = True
                            indeces = extract_numbers(block_text)
                            sub_index = int(indeces[0])
                            upper_index = int(indeces[-1])
                            multi_context_question_set = [index for index in range(sub_index, upper_index + 1)]
                            continue
                        if multi_context:
                            # If it's a multi-context question, save the text and continue
                            if multi_context_question_text == "":
                                multi_context_question_text = block_text
                            else:   
                                multi_context_question_text += " " + block_text
                            


                        if f"{exam_year}-{exam_subtype}/" in block_text:
                            if "\n" in block_text and len(block_text.split("\n")) > 1:
                                if block_text.split("\n")[1] != "":
                                    current_section = block_text.split("\n")[1]
                            continue
                        
                        if is_question_index(block_text):
                            # Close the multi-context question if it was open
                            if multi_context:
                                multi_context = False



                            # If it's a new question, create a new dictionary
                        
                            new_question = {}
                            new_question["year"] = exam_year
                            new_question["exam"] = exam_type
                            new_question["subtype"] = exam_subtype
                            new_question["section"] = current_section
                            new_question["question_number"] = int(block_text.split(".")[0])
                            new_question["page"] = page_num
                            new_question["q_language"] = "tr"
                            new_question["question"] = ""
                            new_question["has_image"] = False
                            new_question["max_image_heigth"] = 0
                            new_question["max_image_width"] = 0
                            new_question["passage"] = ""
                            new_question["choices"] = []
                            new_question["answer"] = ""
                            new_question["image_count"] = 0
                            current_question = new_question
                            question_list.append(current_question)

                            if multi_context_question_text != "":
                                # If there was a multi-context question, add it to the current question
                                if new_question["question_number"] in multi_context_question_set:
                                    current_question["passage"] = multi_context_question_text
                                    if new_question["question_number"] == multi_context_question_set[-1]:
                                        multi_context_question_text = ""
                                        multi_context_question_set = []


                            continue

                        if current_question:
                            # Append the text to the current question - FIXED HERE
                            if current_question["question"] == "":
                                current_question["question"] = block_text  # Actually save the first text block
                            else:
                                current_question["question"] += " " + block_text

                    elif block[7] == 1:
                        # If it's an image block, add the image reference to the current question
                        if current_question:
                            current_question["has_image"] = True
                            current_question["image_count"] += 1
                            current_question["max_image_heigth"] = max(current_question["max_image_heigth"], block[5][0])
                            current_question["max_image_width"] = max(current_question["max_image_width"], block[5][1])

                            if current_question["question"] == "":
                                current_question["question"] = f"<page_{page_num}_img_{block[6]}.{block[5][2]}> "
                            else:
                                current_question["question"] += f" <page_{page_num}_img_{block[6]}.{block[5][2]}> "


        else: ## If it's the answer key, we don't need to process the blocks
            pass




# Order the choices as they are mixed

# everything after the '?' is a choice

for question in question_list:

    parts = []

    # Split the question text into parts
    parts = split_question_choices(question["question"])
    
    # If there are multiple parts, the first part is the question and the rest are choices
    if len(parts) > 1:
        question_text = parts[0] + "?"
        
        i = 1

        while i < len(parts):
            if has_choices(parts[i], expected_choices=5):

                choices_text = parts[i]
                break
            
            i += 1


        
        # Split the choices by line breaks and remove empty strings
        choices = [choice.strip() for choice in choices_text.split("\n") if choice.strip()]
        
        # Assign the question text and choices to the dictionary
        question["question"] = question_text.strip()
        question["choices_raw"] = choices_text
        question["choices"] = create_choice_list(choices_text)




## Test if the choices extrated correctly

for question in question_list:

    if not has_choices(question["choices_raw"], expected_choices=5):
        print(question["section"], question["question_number"])

    if question["choices"] == -1:
        print(f"Section: {question["section"]}, {question["question_number"]}")

# Primt the question list as json
json_data = json.dumps(question_list, ensure_ascii=False, indent=4)


# Save the JSON data to a file
with open(f"../data/{exam_name}_questions.json", "w", encoding="utf-8") as json_file:
    json_file.write(json_data)


        

            



Choices are not extracted correctly 1
Section: TÜRKÇE TESTİ, 11
Section: SOSYAL BİLİMLER TESTİ, 4
Section: SOSYAL BİLİMLER TESTİ, 17
Section: TEMEL MATEMATİK TESTİ, 8
Section: TEMEL MATEMATİK TESTİ, 10
Section: TEMEL MATEMATİK TESTİ, 13
Section: TEMEL MATEMATİK TESTİ, 15
Section: TEMEL MATEMATİK TESTİ, 16
Section: FEN BİLİMLERİ TESTİ, 5
Section: FEN BİLİMLERİ TESTİ, 8
Section: FEN BİLİMLERİ TESTİ, 10


In [172]:
import re

def create_choice_list(choices):

        res = re.split(r'\n', choices)


        #remove empty strings
        res = [i for i in res if len(i.strip()) > 0]

        if len(res) < 10:
            res = re.split(r'[\s]', choices)
            #remove empty strings again
            res = [i for i in res if len(i.strip()) > 0]

        elif len(res) > 10:
            #remove empty strings again
            

            new_res = [""] * 10
            new_res_ind = -1
            for index in range(len(res)):
                first_char = res[index].lstrip()[0]
                if first_char.isupper() or first_char in ["<"]:

                    new_res_ind += 1
                    if new_res_ind > 9:
                        #print("\033[31mChoices are not extracted correctly 4\033[0m")
                        return -1
                    new_res[new_res_ind] = res[index]
                else:
                    new_res[new_res_ind] += " " + res[index]
                    
                    

            res = new_res

        
        
        #if there is problem notice the user
        if len(res) != 10:
            print("\033[31mChoices are not extracted correctly 1\033[0m")
            return -1

     
        result = [""] * 10

        choice_ind = 0
        non_choice_ind = 1

        for i in range(10):

            # Check if the string matches the pattern for choices (A), B), C), D), E))

            if bool(re.match(r'\s*[ABCDE]\)', res[i])):
                if choice_ind > 9:
                    #print("\033[31mChoices are not extracted correctly 2\033[0m")
                    return -1
                result[choice_ind] = res[i]
             
                choice_ind += 2
            else:
                if non_choice_ind > 9:
                    #print("\033[31mChoices are not extracted correctly 3\033[0m")
               
                    return -1
                result[non_choice_ind] = res[i]
                
                non_choice_ind += 2

        final_result = [f"{result[i]} {result[i + 1]}" for i in range(0, len(result), 2)]

        return final_result






choices_text = "<page_25_img_6.png>  <page_25_img_5.png>  A)\nB)\n <page_25_img_7.png>  <page_25_img_8.png>  C)\nD)\n <page_25_img_9.png>  E)\n <page_25_img_10.png> "


create_choice_list(choices_text)

Choices are not extracted correctly 1


-1

# Use Pandas to proccess the JSON

In [169]:
import pandas as pd
import json

from pathlib import Path

exam_name = "2024_YKS_TYT"

# Load the JSON data from the file

json_path = Path("..") / "data" / f"{exam_name}_questions.json"

with open(json_path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

# Convert the JSON data to a DataFrame

df = pd.DataFrame(data)

df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              125 non-null    object 
 1   exam              125 non-null    object 
 2   subtype           125 non-null    object 
 3   section           125 non-null    object 
 4   question_number   125 non-null    int64  
 5   page              125 non-null    int64  
 6   q_language        125 non-null    object 
 7   question          125 non-null    object 
 8   has_image         125 non-null    bool   
 9   max_image_heigth  125 non-null    float64
 10  max_image_width   125 non-null    float64
 11  passage           125 non-null    object 
 12  choices           125 non-null    object 
 13  answer            125 non-null    object 
 14  image_count       125 non-null    int64  
 15  choices_raw       125 non-null    object 
dtypes: bool(1), float64(2), int64(3), object(10)

# Process the images